In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
from os import listdir, getcwd
from os.path import isfile, join, exists
from math import radians,sin
from tqdm import tqdm

In [2]:
def great_circle_list(clon,clat,XLON,XLAT):
    """
    clon and clat are the target point (in degrees) to be compared to every point in XLON and XLAT
    XLON and XLAT should be the list of points that make up the front
    Out put is a list of Great Circle Distances (GCD) from each point to the target point in meters
    """
    rearth = 6371000 #meters
    clon,clat = map(radians,[clon,clat])
    XLON,XLAT = map(np.radians,[XLON,XLAT])
    
    return rearth * ( np.arccos(sin(clat) * np.sin(XLAT) + np.cos(clat) * np.cos(XLAT) * np.cos(clon - XLON)) )

def GCD_m2deg(GCD):
    rearth = 6371000 #meters
    return (GCD / (2*np.pi*rearth)) * 360

def GCD_deg2m(GCD):
    rearth = 6371000 #meters
    return (GCD / 360) * (2*np.pi*rearth)

def GetFrontCoordinates(lines):
    FRONT_coords = []
    number = 0
    while number < len(lines):
        if len(lines[number])<=2:
            number+=1
        elif lines[number].split()[0] in ['WARM','STNRY','COLD','OCFNT']:
            coordlist = [lines[number].split()[0],lines[number].split()[1]]
            if lines[number+1][0].isdigit():
                front_data = lines[number].split() + lines[number+1].split()
                number+=1
            else:
                front_data = lines[number].split()
            for coords in front_data[2:]:
                coordlist.append([int(coords[:2]),int(coords[2:])])
            FRONT_coords.append(coordlist)
            number+=1
        else:
            number+=1
            
    return FRONT_coords

def ExtractPlotData(frt):
    lats = []
    lons = []
    for co in frt[2:]:
        lats.append(co[0])
        lons.append(-1*co[1])
        
    if frt[0] == 'WARM':
        typ = 'red'
    elif frt[0] == 'STNRY':
        typ = 'green'
    elif frt[0] == 'COLD':
        typ = 'blue'
    elif frt[0] == 'OCFNT':
        typ = 'violet'
        
    return lons,lats,typ

def GCD_Point2Front_AtTime(clon,clat,frts):
    mindist = []
    for frt in frts:
        lons,lats,_ = ExtractPlotData(frt)
        gcds = GCD_m2deg(great_circle_list(clon,clat,np.array(lons),np.array(lats)))
        mindist.append(np.min(gcds))
        # WHAT IF WE ALSO CHECK THE MID POINTS
        mlons = []
        mlats = []
        for gg in range(len(lons)-1):
            mlons.append((lons[gg]+lons[gg+1])/2)
            mlats.append((lats[gg]+lats[gg+1])/2)
        gcds = GCD_m2deg(great_circle_list(clon,clat,np.array(mlons),np.array(mlats)))
        if len(gcds) > 1:
            mindist.append(np.min(gcds))
        
    return min(mindist)

In [3]:
# for YEAR in range(1980,2011):
YEAR = 2009
pdat = '../Data/ERA5_'+str(YEAR)+'_tp.nc'
pcip = xr.open_dataset(pdat)

pcip = pcip['tp'].resample(time='3H').sum()
print(pcip['time'].dt.hour)

<xarray.DataArray 'hour' (time: 2920)>
array([ 0,  3,  6, ..., 15, 18, 21])
Coordinates:
  * time     (time) datetime64[ns] 2009-01-01 ... 2009-12-31T21:00:00


In [4]:
percent = np.nanpercentile(np.where(pcip>0,pcip,np.nan),q=95,axis=0)

In [5]:
isPEX = np.where(pcip>np.broadcast_to(percent,pcip.shape),1,0)
Events,elats,elons = np.where(isPEX==1)

In [6]:
D2F = np.ones(Events.shape) * 999
noFront = 0
for lm in tqdm(range(len(D2F))):
    year = pcip['time'].dt.year.values[Events[lm]]
    month= pcip['time'].dt.month.values[Events[lm]]
    day= pcip['time'].dt.day.values[Events[lm]]
    hour= pcip['time'].dt.hour.values[Events[lm]]
    p2f = 'WPC_CODSUS/lores/'+str(year)+'/KWBCCODSUS_'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_'+str(hour).zfill(2)+'00'
    if exists(p2f):
        with open(p2f) as f:
            lines = f.readlines()
        f.close()
        FC = GetFrontCoordinates(lines)
        if len(FC) == 0:
            noFront += 1
        else:
            D2F[lm] = GCD_Point2Front_AtTime(pcip['longitude'].values[elons[lm]],pcip['latitude'].values[elats[lm]],FC)
    else:
        noFront += 1
        continue


100%|███████████████████████████████████████████████████████████| 306221/306221 [11:58<00:00, 426.33it/s]


In [3]:
flist = []
for YEAR in range(2003,2011):
    flist.append('../Data/ERA5_'+str(YEAR)+'_tp.nc')
pcip = xr.open_mfdataset(flist,concat_dim='time')
pcip = pcip['tp'].resample(time='3H').sum()
percent = np.nanpercentile(np.where(pcip>0,pcip,np.nan),q=90,axis=0)
isPEX = np.where(pcip>np.broadcast_to(percent,pcip.shape),1,0)
Events,elats,elons = np.where(isPEX==1)

D2F = np.ones(Events.shape) * 999
noFront = 0
for lm in tqdm(range(len(D2F))):
    year = pcip['time'].dt.year.values[Events[lm]]
    month= pcip['time'].dt.month.values[Events[lm]]
    day= pcip['time'].dt.day.values[Events[lm]]
    hour= pcip['time'].dt.hour.values[Events[lm]]
    p2f = 'WPC_CODSUS/lores/'+str(year)+'/KWBCCODSUS_'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_'+str(hour).zfill(2)+'00'
    if exists(p2f):
        with open(p2f) as f:
            lines = f.readlines()
        f.close()
        FC = GetFrontCoordinates(lines)
        if len(FC) == 0:
            noFront += 1
        else:
            D2F[lm] = GCD_Point2Front_AtTime(pcip['longitude'].values[elons[lm]],pcip['latitude'].values[elats[lm]],FC)
    else:
        noFront += 1
        continue

  0%|▏                               | 14676/3376078 [00:52<3:19:45, 280.45it/s]


KeyboardInterrupt: 